In [12]:
%load_ext nb_black
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import pprint
from config import app_key
import pymongo
from time import time, sleep

url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
parameters = {"start": 1, "limit": 1000, "convert": "USD"}
headers = {
    "Accept": "application/json",
    "X-CMC_PRO_API_KEY": app_key,
}

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [16]:
app_key

{'X-CMC_PRO_API_KEY': 'f5edd51c-defa-48b5-9af8-60226629f498'}

<IPython.core.display.Javascript object>

In [7]:
session = Session()
session.headers.update(headers)
data_retrieved = []
pp = pprint.PrettyPrinter(width=10, compact=True)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    pp.pprint(data)
    data_retrieved.append(data)

except (ConnectionError, Timeout, TooManyRedirects) as error:
    data_retrieved(error)


InvalidHeader: Value for header {X-CMC_PRO_API_KEY: {'X-CMC_PRO_API_KEY': 'f5edd51c-defa-48b5-9af8-60226629f498'}} must be of type str or bytes, not <class 'dict'>

<IPython.core.display.Javascript object>

In [3]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.cryptoCurrency_db
collection = db.items


<IPython.core.display.Javascript object>

In [4]:
for item in data_retrieved:

    collection.insert_one(item)


NameError: name 'data_retrieved' is not defined

<IPython.core.display.Javascript object>

In [33]:
docs = pd.DataFrame(columns=[])
items = db.items.find()
for num, doc in enumerate(items):
    doc["_id"] = str(doc["_id"])
    doc_id = doc["_id"]
    series_obj = pd.Series(doc, name=doc_id)
    docs = docs.append(series_obj)


<IPython.core.display.Javascript object>

In [34]:
data_df = pd.DataFrame(docs["data"][0])
data_df["date_added"] = pd.to_datetime(data_df["date_added"])
data_df["year"] = pd.DatetimeIndex(data_df["date_added"]).year
data_df.head(2)


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote,year
0,1,Bitcoin,BTC,bitcoin,9357,2013-04-28 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.902007e+07,1.902007e+07,None,1,NaN,NaN,2022-04-23T19:42:00.000Z,"{'USD': {'price': 39732.432568576856, 'volume_...",2013
1,1027,Ethereum,ETH,ethereum,5661,2015-08-07 00:00:00+00:00,"[mineable, pow, smart-contracts, ethereum-ecos...",NaN,1.204986e+08,1.204986e+08,None,2,NaN,NaN,2022-04-23T19:41:00.000Z,"{'USD': {'price': 2951.497484058154, 'volume_2...",2015


<IPython.core.display.Javascript object>

In [35]:
df = pd.DataFrame.from_dict(data_df)
df.head(2)
df["date_added"] = pd.to_datetime(df["date_added"])
df["year"] = pd.DatetimeIndex(df["date_added"]).year


<IPython.core.display.Javascript object>

In [36]:
df = df["quote"].apply(pd.Series)
df.head(2)


,USD
0,"{'price': 39732.432568576856, 'volume_24h': 16..."
1,"{'price': 2951.497484058154, 'volume_24h': 971..."


<IPython.core.display.Javascript object>

In [37]:
df = df.USD.apply(pd.Series)
df.head(2)


,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,39732.432569,1.665930e+10,-47.3935,-0.034540,0.003928,-1.248341,-9.733043,4.534797,13.781847,7.557136e+11,40.8958,8.343811e+11,2022-04-23T19:42:00.000Z
1,2951.497484,9.715132e+09,-49.7670,-0.057011,-0.887322,-2.507274,-5.258144,12.748583,22.701376,3.556514e+11,19.2455,3.556514e+11,2022-04-23T19:41:00.000Z


<IPython.core.display.Javascript object>

In [38]:
final_df = pd.concat([data_df, df], axis=1)
final_df = final_df.drop(columns="quote")
final_df


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,last_updated
0,1,Bitcoin,BTC,bitcoin,9357,2013-04-28 00:00:00+00:00,"[mineable, pow, sha-256, store-of-value, state...",21000000.0,1.902007e+07,1.902007e+07,...,-0.034540,0.003928,-1.248341,-9.733043,4.534797,13.781847,7.557136e+11,40.8958,8.343811e+11,2022-04-23T19:42:00.000Z
1,1027,Ethereum,ETH,ethereum,5661,2015-08-07 00:00:00+00:00,"[mineable, pow, smart-contracts, ethereum-ecos...",NaN,1.204986e+08,1.204986e+08,...,-0.057011,-0.887322,-2.507274,-5.258144,12.748583,22.701376,3.556514e+11,19.2455,3.556514e+11,2022-04-23T19:41:00.000Z
2,825,Tether,USDT,tether,32249,2015-02-25 00:00:00+00:00,"[payments, stablecoin, asset-backed-stablecoin...",NaN,8.311420e+10,8.571395e+10,...,0.002965,0.004818,0.002284,-0.021181,-0.043373,-0.016517,8.313830e+10,4.5004,8.573880e+10,2022-04-23T19:41:00.000Z
3,1839,BNB,BNB,bnb,813,2017-07-25 00:00:00+00:00,"[marketplace, centralized-exchange, payments, ...",165116760.0,1.632770e+08,1.632770e+08,...,0.016126,-0.776389,-1.931158,-2.120099,9.499423,10.934343,6.633072e+10,3.5902,6.707813e+10,2022-04-23T19:41:00.000Z
4,3408,USD Coin,USDC,usd-coin,3801,2018-10-08 00:00:00+00:00,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,4.989235e+10,4.989235e+10,...,0.045592,-0.006661,0.045157,0.066560,-0.003620,0.100152,4.991087e+10,2.7006,4.991087e+10,2022-04-23T19:41:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2576,Tokenomy,TEN,tokenomy,5,2018-03-10 00:00:00+00:00,[marketplace],NaN,2.000000e+08,2.000000e+08,...,-0.162602,-2.346419,-8.465532,5.240591,-6.582353,-1.769172,8.486524e+06,0.0005,8.486524e+06,2022-04-23T19:41:00.000Z
996,8161,Shopping,SPI,shopping,19,2021-01-02 00:00:00+00:00,"[e-commerce, bnb-chain]",NaN,9.195561e+05,1.000000e+06,...,-0.179282,-1.308004,4.629906,13.567833,3.978736,-17.350695,8.484869e+06,0.0000,9.227136e+06,2022-04-23T19:41:00.000Z
997,3336,IQeon,IQN,iqeon,11,2018-09-24 00:00:00+00:00,"[gaming, entertainment]",5491860.0,5.491860e+06,5.491860e+06,...,-0.396283,0.472776,-3.777212,-42.327275,-49.506689,-37.459632,8.440723e+06,0.0000,8.440723e+06,2022-04-23T19:41:00.000Z
998,3965,TouchCon,TOC,touchcon,4,2019-05-24 00:00:00+00:00,[],NaN,1.584001e+08,9.900000e+08,...,-0.057011,-1.704184,-17.614995,164.310742,481.460549,818.987483,8.438691e+06,0.0000,5.274178e+07,2022-04-23T19:41:00.000Z


<IPython.core.display.Javascript object>

In [39]:
collection.insert_many(final_df.to_dict("records"))


/var/folders/4p/rh50sq7s6hv5kgschbryjvt80000gn/T/ipykernel_66710/1793185836.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  collection.insert_many(final_df.to_dict('records'))


<IPython.core.display.Javascript object>

In [1]:
collection

NameError: name 'collection' is not defined